To web scrap data from Estes Park Weather website. 
Source:https://www.estesparkweather.net/archive_reports.php?date=
Libraries used: BeautifulSoup, requests, pandas, numpy, datetime

In [ ]:
#install required libraries
#!pip install bs4
#!pip install requests

In [240]:
#import required libraries
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [73]:
# For viewing the html script of the website of a month
date="202203"
url = 'https://www.estesparkweather.net/archive_reports.php?date='+date
source = requests.get(url)
page=bs(source.content,'html.parser')
#print(page)
table_list=page.find_all('table')
#print(table_list)

In [241]:
#For getting the start-year/month/date and end-year/month/date

stdate="2009-01-01"
eddate="2018-10-28"
startdate=stdate.replace('-','')
enddate=eddate.replace('-','')

#Validating and formatting the given start and end dates

if len(startdate)==8 and len(enddate)==8:
    
    #Separating the years
    startyear=int(startdate[:4])
    endyear=int(enddate[:4])+1
    
    #Separating the months
    startmonth=int(startdate[4:6])
    endmonth=int(enddate[4:6])
    if startmonth>=12 and endmonth>=12:
        print("Provide a valid month")
    
    #Separating the dates
    startdate=int(startdate[6:])
    enddate=int(enddate[6:])
    if startdate>=31 and endmonth>=31:
        print("Provide a valid date")
        
else:
    print("Provide a valid date format")
    
# Creating an empty dataframe and dictionary
df=pd.DataFrame(columns = ['Date',
                           'Average temperature (°F)',
                           'Average humidity (%)',
                           'Average dewpoint (°F)',
                           'Average barometer (in)',
                           'Average windspeed (mph)',
                           'Average gustspeed (mph)',
                           'Average direction (°deg)',
                           'Rainfall for month (in)',
                           'Rainfall for year (in)',
                           'Maximum rain per minute',
                           'Maximum temperature (°F)',
                           'Minimum temperature (°F)',
                           'Maximum humidity (%)',
                           'Minimum humidity (%)',
                           'Maximum pressure',
                           'Minimum pressure',
                           'Maximum windspeed (mph)',
                           'Maximum gust speed (mph)',
                           'Maximum heat index (°F)'])

dict_weather={'Average temperature (°F)':0, 
              'Average humidity (%)':0,
              'Average dewpoint (°F)':0,
              'Average barometer (in)':0,
              'Average windspeed (mph)':0,
              'Average gustspeed (mph)':0,
              'Average direction (°deg)':0,
              'Rainfall for month (in)':0,
              'Rainfall for year (in)':0,
              'Maximum rain per minute':0,
              'Maximum temperature (°F)':0,
              'Minimum temperature (°F)':0,
              'Maximum humidity (%)':0,
              'Minimum humidity (%)':0,
              'Maximum pressure':0,
              'Minimum pressure':0,
              'Maximum windspeed (mph)':0,
              'Maximum gust speed (mph)':0,
              'Maximum heat index (°F)':0}
        

# Getting URL set and scraping data for given start and end years
url_template="http://www.estesparkweather.net/archive_reports.php?date={}{}"
for year in range(startyear,endyear):
    for month in range(1,13):
        if month <10:
            month="0"+str(month)
        else:
            month=month
        url= url_template.format(year,month)
        source = requests.get(url)
        page=bs(source.content,'html.parser')
        #print(page)
               
# Getting the entire table list for given years
        table_list=page.find_all('table')
        table_list=table_list[:(len(table_list))-9]
        #print(table_list)

# Getting the info for every day 
        for table in table_list:
            #print(table)
            table_content=table.find_all('tr',class_=["table-top","column-light","column-dark"])
            #print(table_content)
            data_per_day=[]
            
            #Appending a day's info to list
            for info in table_content:
                info=info.text.strip('\n')
                #print(info)
                data_per_day.append(info)
                
                #Creating date format 
                months={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',
                        'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
                dfmonth=months.get((data_per_day[0].split()[0]))
                dfdate=data_per_day[0].split()[1]
                if dfdate.isnumeric() == True:
                    if int(dfdate)<10:
                        dfdate="0"+str(dfdate)
                        #print(dfdate)
                else:
                    dfdate=dfdate
                    #print(dfdate)
                date=str(year)+"-"+str(dfmonth)+"-"+str(dfdate)
                
            #Updating the info to dictionary    
            dict_weather={
                'Date':date,
                'Average temperature (°F)':data_per_day[1].split()[2].strip("°F"),#(° ---> Alt+0176) 
                'Average humidity (%)':data_per_day[2].split()[2].strip("%"),
                
                'Average dewpoint (°F)':data_per_day[3].split()[2].strip("°F"),
                'Average barometer (in)':data_per_day[4].split()[2],
                
                'Average windspeed (mph)':data_per_day[5].split()[2],
                'Average gustspeed (mph)':data_per_day[6].split()[2],
                
                'Average direction (°deg)':data_per_day[7].split()[2].strip("°"),
                'Rainfall for month (in)':data_per_day[8].split()[3],
                
                'Rainfall for year (in)':data_per_day[9].split()[3],
                'Maximum rain per minute':data_per_day[10].split()[4],
                
                'Maximum temperature (°F)':data_per_day[11].split()[2].strip("°F"),
                'Minimum temperature (°F)':data_per_day[12].split()[2].strip("°F"),
                
                'Maximum humidity (%)':data_per_day[13].split()[2].strip("%"),
                'Minimum humidity (%)':data_per_day[14].split()[2].strip("%"),
                
                'Maximum pressure':data_per_day[15].split()[2],
                'Minimum pressure':data_per_day[16].split()[2],
                
                'Maximum windspeed (mph)':data_per_day[17].split()[2],
                'Maximum gust speed (mph)':data_per_day[18].split()[3],
                
                'Maximum heat index (°F)':data_per_day[19].split()[3].strip("°F")}
            
            #Appending the data from dictionary to respective dataframe columns
            df=df.append(dict_weather,ignore_index=True)
            
            #Setting date column as index
            df.set_index("Date")
#display(df)  

In [268]:
# Selecting from the dataframe for given date range
s=df[df['Date']==stdate].index.values.item()
e=df[df['Date']==eddate].index.values.item()
final_df=df.iloc[s:e]
#display(final_df)

In [272]:
#Removing the records with invalid date entires
final_df=final_df[~final_df.Date.str.contains("None")]
#display(final_df)
#final_df.shape

In [275]:
#final_df.info()

In [276]:
#Converting the data types of the column

#Creating list of columns
column_list=final_df.columns.tolist()

#Column "Date" is skipped as it in date format
for column in column_list[1:]:
    #Converting the columns to thier respective datatypes
    final_df[column]=pd.to_numeric(final_df[column])
final_df['Date']=pd.to_datetime(final_df['Date'])
#final_df.info()

In [277]:
#final_df.shape
#final_df.isna().sum()
#final_df.isnull().sum()
#print(final_df.nunique())
#dup_dates = final_df.pivot_table(columns=['Date'], aggfunc='size')
#final_df[final_df.duplicated('Date',keep=False)]

In [274]:
#Converting the dataframe to excel and csv file
final_df.to_excel("weather.xlsx")
final_df.to_csv("weather.csv")

In [278]:
#Saving the dataframe as pickle file
import pickle
with open("dataframe.pk", "wb") as file:
    pickle.dump(final_df, file)

In [279]:
#Reading the pickle file
data = pd.read_pickle('dataframe.pk')
#display(data)

In [280]:
#count of missing records
date=pd.date_range(stdate,eddate)
count_gn_dates=len(date)
count_df_dates=len(final_df)
#print(abs(count_gn_dates-count_df_dates))